In [1]:
# Install dependencies
import random
from tile import Tile
from plate import Plate
from typing import List

In [28]:
yellow = "yellow"
red = "red"
blue = "blue"
green = "green"
black = "black"
# Test version of Board class
class Space():
    """Space on score board"""

    def __init__(self, color: str = None, minus_value: int = None):
        self.tile = None
        self.color = color
        self.minus_value = None

    @property
    def _tile(self, _tile: str):
        self.tile = _tile

    @property
    def _minus_value(self, _minus_value: int):
        self.minus_value = _minus_value
        
class Board():
    """5x5 Playing board to track game progress"""

    def __init__(self, name: str):
        self.game_over = False
        self.name = name
        self.spaces = [
            [Space(yellow), Space(blue), Space(green), Space(red), Space(black)],
            [Space(black), Space(yellow), Space(blue), Space(green), Space(red)],
            [Space(red), Space(black), Space(yellow), Space(blue), Space(green)],
            [Space(green), Space(red), Space(black), Space(yellow), Space(blue)],
            [Space(blue), Space(green), Space(red), Space(black), Space(yellow)],
            ]

    def check_rows(self):
        """
        Check if the end-game condition has been met. Game ends if
        any row on a Board is filled with tiles
        """
        for i in range(len(self.spaces)):
            has_tiles = [self.spaces[i][j].tile for j in range(len(self.spaces))]
            if all(has_tiles):
                self.game_over = True
        return

class Factory_row():
    """ 
    Define a row of the Factory board, and track which color options are playable
    """

    def __init__(self, name: str, num_spaces: int):
        self.name = name
        self.num_spaces = num_spaces
        self.spaces = [Space() for i in range(self.num_spaces)]
        self.scoring_colors = [yellow, red, green, blue, black]
        self.playable_colors = [yellow, red, green, blue, black]
        self.playable = True
        self.row_color = None

    def place_tiles(self, tiles: List[Tile]):
        """Place tiles in factory row, and adjust the options for that row accordingly"""
        self.row_color = tiles[0].color
        for tile in tiles:
            for space in self.spaces:
                if not space.tile:
                    space.tile = tile
    
    def select_playable_color(self):
        """Randomly select a color"""
        options = random.shuffle(self.playable_colors)
        selected_color = options.pop()
        self.playable_colors = [selected_color]

    def check_complete(self):
        """Check if all spaces in the row have tiles"""
        has_tiles = [space.tile for space in self.spaces]
        if all(has_tiles):
            self.spaces = [Space() for i in range(self.num_spaces)]
            self.scoring_colors.remove(self.row_color)
            self.row_color = None

class Factory():
    """
    Board where tiles are collected by players before being moved
    to the scoring board.
    """

    def __init__(self):
        self.floor = Floor()
        self.row_1 = Factory_row(name="row_1", num_spaces=1)
        self.row_2 = Factory_row(name="row_2", num_spaces=2)
        self.row_3 = Factory_row(name="row_3", num_spaces=3)
        self.row_4 = Factory_row(name="row_4", num_spaces=4)
        self.row_5 = Factory_row(name="row_5", num_spaces=5)

        self.playable_rows = [self.row_1, self.row_2, self.row_3, self.row_4, self.row_5]

    def select_playable_row(self):
        """Randomly select a playable row"""

        if self.playable_rows:
            return random.choice(self.playable_rows)
        else:
            return self.floor
            

class Floor():
    """
    Minus section of a player's Factory
    """

    def __init__(self):
        self.spaces = [
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=2),
            Space(minus_value=2),
            Space(minus_value=3),
            Space(minus_value=3),
            ]
        self.discard = []

    def reset(self):
        """Reset Floor to original state"""
        for space in self.spaces:
            if space.tile:
                self.discard.append(space.tile)

        self.spaces = [
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=2),
            Space(minus_value=2),
            Space(minus_value=3),
            Space(minus_value=3),
            ]

class Player():
    """
    Represent a player in the game
    """

    def __init__(self, name=str):
        self.name = name
        self.factory = Factory()
        self.board = Board("player_1")


class Table():
    """ 
    Represents where left over tiles from plates go
    """
    
    def __init__(self, n_plates: int = 5):
        colors = [red, green, blue, yellow, black]
        self.bag = [Tile(color) for color in colors for _ in range(20)]
        self.leftover_tiles = [Tile(color=None, is_first=True)]
        playable_colors = []

        self.plates = []
        for i in range(n_plates):
            plate_name = f"plate_{i}"
            new_plate = Plate(plate_name, 0, 0, 0, 0)
            # Fill plates with 4 tiles
            for _ in range(4):
                # Shuffle bag and remove last tile in bag
                random.shuffle(self.bag)
                rand_tile = self.bag.pop()
                new_plate.add_tile(rand_tile)
                playable_colors.append(rand_tile.color)
            self.plates.append(new_plate)
        self.playable_colors = set(playable_colors)

    def update_playable_colors(self):
        """Update the unique list of colors of all tiles on Table"""

        leftover_tile_colors = [tile.color for tile in self.leftover_tiles]
        if None in leftover_tile_colors:
            leftover_tile_colors.remove(None)

        plate_tile_colors = []
        for plate in self.plates:
            for tile in plate.tiles:
                plate_tile_colors.append(tile.color)
        self.playable_colors = set(leftover_tile_colors + plate_tile_colors)


    def reset_table(self, floors: List[Floor]):
        # If the bag does not have enough tiles to fill the plates
        # Place all discarded tiles from
        if len(self.bag) < 20:
            for floor in floors:
                for tile in floor.discard:
                    self.bag.append(tile)

        self.leftover_tiles = [Tile(color=None, is_first=True)]
        self.playable_colors = []
        self.plates = []
        for i in range(n_plates):
            plate_name = f"plate_{i}"
            new_plate = Plate(plate_name, 0, 0, 0, 0)
            # Fill plates with 4 tiles
            for _ in range(n_plate_tiles):
                # Shuffle bag and remove last tile in bag
                random.shuffle(self.bag)
                rand_tile = self.bag.pop()
                new_plate.add_tile(rand_tile)
                self.playable_colors.append(rand_tile.color)
            self.plates.append(new_plate)



In [29]:
# Game conditions
n_players = 2
n_plates = 5 + (n_players -2) * 2
n_plate_tiles = 4

# Create game elements
colors = ["green", "yellow", "blue", "red", "black"]

player_1 = Player("kaci")
player_2 = Player("avi")

# Play a game to completion
#while not player_1.board.game_over and not player_2.board.game_over:
table = Table(n_plates=5)
print(table.playable_colors)


    # # Player_1's turn
    # # Choose factory row to try to fill
    # in_play_row = player_1.factory.select_playable_row()

    # # Check if there are any available tiles on plates or on table that are 
    # # in the playable_colors of the selected in_play_row
    # if any(color in in_play_row.playable_colors for color in set(plates_playable_colors + ))

    

    
    



{'blue', 'yellow', 'green', 'black', 'red'}


In [12]:
test1 = ["hello", "no", "you"]
test2 = ["no"]

test1.remove(None)

print(test1)

ValueError: list.remove(x): x not in list

In [22]:
test = [Tile(color) for color in colors for _ in range(5)]
test = test